<a href="https://colab.research.google.com/github/matsunagalab/mdzen/blob/main/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
## Step 3: Visualize Final Structure

View the final frame of the simulation with water molecules wrapped into the periodic box.

---
## Setup: Install Dependencies

**First time only** - This installs AmberTools, OpenMM, and other required packages.

⏱️ Takes ~2-4 minutes. You can continue reading while it runs.

In [ ]:
#@title ▶️ Run Setup (click to expand code)
import sys
import os
import time
import socket
import subprocess

IN_COLAB = 'google.colab' in sys.modules

#==============================================================================
# API Key Configuration
#==============================================================================
# os.environ['ANTHROPIC_API_KEY'] = 'sk-ant-...'
#==============================================================================

def load_dotenv():
    for env_path in ['./.env', '../.env', '/content/.env', '/content/mdzen/.env']:
        try:
            with open(env_path) as f:
                for line in f:
                    line = line.strip()
                    if line and not line.startswith('#') and '=' in line:
                        key, value = line.split('=', 1)
                        os.environ[key.strip()] = value.strip().strip('"').strip("'")
            return True
        except FileNotFoundError:
            continue
    return False

load_dotenv()

if IN_COLAB:
    try:
        from google.colab import userdata
        for k in ['ANTHROPIC_API_KEY', 'OPENAI_API_KEY', 'GOOGLE_API_KEY']:
            try:
                v = userdata.get(k)
                if v: os.environ[k] = v
            except: pass
    except: pass

#==============================================================================
# Model Auto-Detection
#==============================================================================
# Default models (cheap/fast) for each provider
DEFAULT_MODELS = {
    'anthropic': ('anthropic:claude-haiku-4-5-20251001', 'anthropic:claude-haiku-4-5-20251001'),
    'openai': ('openai:gpt-4o-mini', 'openai:gpt-4o-mini'),
    'google': ('google:gemini-2.0-flash', 'google:gemini-2.0-flash'),
}

# Detect available API key
detected = None
for k, p in [('ANTHROPIC_API_KEY', 'anthropic'), ('OPENAI_API_KEY', 'openai'), ('GOOGLE_API_KEY', 'google')]:
    if os.environ.get(k):
        detected = p
        print(f"✓ {k}")
        break

if detected:
    clarification_model, setup_model = DEFAULT_MODELS[detected]
    os.environ['MDZEN_CLARIFICATION_MODEL'] = clarification_model
    os.environ['MDZEN_SETUP_MODEL'] = setup_model
    model_name = clarification_model.split(':')[1]
    print(f"✓ Default model: {model_name}")
else:
    print("⚠️ No API key found!")
    print("   Set one of: ANTHROPIC_API_KEY, OPENAI_API_KEY, GOOGLE_API_KEY")

#==============================================================================
# 共通設定: MCP サーバー起動ロジック
#==============================================================================
MCP_SERVERS = [
    ("research_server.py", 8001),
    ("structure_server.py", 8002),
    ("genesis_server.py", 8003),
    ("solvation_server.py", 8004),
    ("amber_server.py", 8005),
    ("md_simulation_server.py", 8006),
]

def check_port(port, timeout=2):
    """Check if a port is listening."""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(timeout)
        result = sock.connect_ex(('localhost', port))
        sock.close()
        return result == 0
    except:
        return False

def start_mcp_servers(python_cmd, server_dir, pythonpath):
    """Start all MCP servers in HTTP mode."""
    procs = []
    for server, port in MCP_SERVERS:
        proc = subprocess.Popen(
            [python_cmd, f"{server_dir}/{server}", "--http", "--port", str(port)],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            env={**os.environ, "PYTHONPATH": pythonpath},
        )
        procs.append((server, port, proc))
    return procs

def wait_for_servers(procs, max_wait=15):
    """Wait for all servers to be ready."""
    print("   Waiting for servers to bind...")
    time.sleep(3)
    waited = 3
    while waited < max_wait:
        ready = sum(1 for _, port, proc in procs if check_port(port) and proc.poll() is None)
        if ready == len(procs):
            return True
        time.sleep(1)
        waited += 1
    return False

def report_server_status(procs):
    """Report server health status."""
    healthy = 0
    failed = []
    for server, port, proc in procs:
        if proc.poll() is not None:
            stderr = proc.stderr.read().decode() if proc.stderr else ""
            err_msg = stderr.split('\n')[-3:-1] if stderr else ["Unknown error"]
            failed.append((server, port, err_msg))
            print(f"   ✗ {server} (:{port}) - CRASHED")
        elif check_port(port):
            print(f"   ✓ {server} (:{port})")
            healthy += 1
        else:
            print(f"   ? {server} (:{port}) - NOT RESPONDING")
            failed.append((server, port, ["Port not listening"]))

    print(f"✓ {healthy}/{len(procs)} MCP servers running (Streamable HTTP)")

    if failed:
        print("\n⚠️ Server startup errors:")
        for server, port, errors in failed:
            print(f"   {server}: {' '.join(errors)[:100]}")

    return healthy

#==============================================================================
# 環境別セットアップ
#==============================================================================
if IN_COLAB:
    start_time = time.time()
    os.chdir('/content')

    # Install Miniforge
    print("📦 Installing Miniforge...")
    !curl -fsSL https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -o /tmp/miniforge.sh
    !bash /tmp/miniforge.sh -b -p /usr/local -u 2>&1 | tail -1
    os.environ["PATH"] = f"/usr/local/bin:{os.environ['PATH']}"
    print(f"✓ Miniforge ({time.time() - start_time:.0f}s)")

    # Install scientific packages
    print("⚗️ Installing scientific packages...")
    !mamba install -y -q openmm pdbfixer parmed ambertools rdkit 2>&1 | tail -2
    print(f"✓ Conda packages ({time.time() - start_time:.0f}s)")

    # Clone repository
    print("📥 Cloning repository...")
    !rm -rf /content/mdzen
    !git clone -q https://github.com/matsunagalab/mdzen.git /content/mdzen
    os.chdir('/content/mdzen')

    # Install pip packages
    print("📦 Installing Python packages (uv)...")
    !pip install -q uv

    CONDA_PYTHON = "/usr/local/bin/python"
    !uv pip install --python {CONDA_PYTHON} -q \
        "litellm>=1.60.0,<1.80.0" \
        anthropic google-genai google-adk \
        "fastmcp>=2.0.0" "mcp[cli]" \
        gradio py3Dmol nest_asyncio \
        mdtraj gemmi pdb2pqr propka dimorphite-dl \
        pubchempy tavily-python

    !uv pip install --python {sys.executable} -q \
        py3Dmol nest_asyncio mdtraj \
        google-adk litellm anthropic \
        "fastmcp>=2.0.0" "mcp[cli]"
    print(f"✓ Pip packages ({time.time() - start_time:.0f}s)")

    # Environment setup
    os.environ["AMBERHOME"] = "/usr/local"
    sys.path.insert(0, '/content/mdzen/src')
    sys.path.insert(0, '/content/mdzen')

    # Verify imports
    try:
        import fastmcp
        from google.adk.runners import Runner
        print("✓ Core packages verified")
    except ImportError as e:
        print(f"⚠️ Import: {e}")

    # Start MCP servers (Colab)
    print("🚀 Starting MCP servers...")
    PYTHON_CMD = "/usr/local/bin/python"
    SERVER_DIR = "/content/mdzen/servers"
    PYTHONPATH = "/content/mdzen/src"

    mcp_procs = start_mcp_servers(PYTHON_CMD, SERVER_DIR, PYTHONPATH)
    wait_for_servers(mcp_procs)
    report_server_status(mcp_procs)

    # Suppress async cleanup warnings
    import logging
    class F(logging.Filter):
        def filter(self, r): return 'cancel scope' not in r.getMessage()
    logging.getLogger('asyncio').addFilter(F())

    print(f"\n✅ Setup complete! ({(time.time() - start_time)/60:.1f} min)")

else:
    # Local Jupyter environment
    sys.path.insert(0, './src')
    load_dotenv()

    # Start MCP servers (Local)
    print("🚀 Starting MCP servers (local)...")
    PYTHON_CMD = sys.executable
    SERVER_DIR = "./servers"
    PYTHONPATH = "./src"

    mcp_procs = start_mcp_servers(PYTHON_CMD, SERVER_DIR, PYTHONPATH)
    wait_for_servers(mcp_procs)
    report_server_status(mcp_procs)

    print("\n✅ Local setup complete!")

---
## Step 1: Describe Your Simulation

Tell the AI what you want to simulate in plain language. The AI will ask clarifying questions to help set up the perfect simulation.

In [ ]:
#@title 🧬 Step 1a: Describe Your Simulation { display-mode: "form" }
#@markdown ### What do you want to simulate?
user_request = "I want to run MD simulation of PDB 1AKE (adenylate kinase) in water at 300K for 0.1 ns" #@param {type:"string"}

#@markdown ---
#@markdown ### Model Selection (optional)
model = "" #@param {type:"string"}
#@markdown > Leave empty for auto-detected default. Examples: `gpt-4o-mini`, `claude-haiku`, `gemini-flash`

#@markdown ---
#@markdown ### Examples (copy one if you like):
#@markdown - `Setup MD for PDB 1AKE in explicit water, 1 ns at 300K`
#@markdown - `Simulate lysozyme (1LYZ) with TIP3P water model`
#@markdown - `Run equilibrium simulation of ubiquitin (1UBQ) at 310K`
#@markdown - `Setup protein-ligand complex from 3HTB for drug binding study`

import sys
import os
import json
import random
import string
from pathlib import Path

# Initialize session
IN_COLAB = 'google.colab' in sys.modules
if 'mdzen_state' not in dir():
    mdzen_state = {
        "session_id": None, 
        "session_dir": None, 
        "user_request": None,
        "clarification_questions": None,
        "user_answers": None,
        "simulation_brief": None, 
        "workflow_outputs": {},
        "model": None,
    }

def init_session():
    job_id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    base_dir = Path("/content/mdzen/outputs") if IN_COLAB else Path("./outputs")
    session_dir = base_dir / f"job_{job_id}"
    session_dir.mkdir(parents=True, exist_ok=True)
    mdzen_state["session_id"] = f"job_{job_id}"
    mdzen_state["session_dir"] = str(session_dir)
    return session_dir

if mdzen_state["session_dir"] is None:
    init_session()

# Save user request
mdzen_state["user_request"] = user_request.strip()

# Handle model selection
def normalize_model_name(model_str):
    """Normalize short model names to full provider:model format."""
    if not model_str or not model_str.strip():
        return None
    model_str = model_str.strip()
    
    # Short aliases
    aliases = {
        "gpt-4o": "openai:gpt-4o",
        "gpt-4o-mini": "openai:gpt-4o-mini",
        "gpt-4": "openai:gpt-4",
        "claude-opus": "anthropic:claude-opus-4-5-20251101",
        "claude-sonnet": "anthropic:claude-sonnet-4-20250514",
        "claude-haiku": "anthropic:claude-haiku-4-5-20251001",
        "gemini-pro": "google:gemini-pro",
        "gemini-flash": "google:gemini-2.0-flash",
    }
    
    if model_str.lower() in aliases:
        return aliases[model_str.lower()]
    elif ":" in model_str:
        return model_str  # Already in provider:model format
    elif model_str.startswith("gpt"):
        return f"openai:{model_str}"
    elif model_str.startswith("claude"):
        return f"anthropic:{model_str}"
    elif model_str.startswith("gemini"):
        return f"google:{model_str}"
    else:
        return model_str

if model.strip():
    normalized = normalize_model_name(model)
    if normalized:
        os.environ['MDZEN_CLARIFICATION_MODEL'] = normalized
        os.environ['MDZEN_SETUP_MODEL'] = normalized
        mdzen_state["model"] = normalized
        
        # Reload config settings
        try:
            from mdzen import config
            config.settings = config.Settings()
        except:
            pass

print("=" * 60)
print("  ✅ Request Received!")
print("=" * 60)
print(f"  📝 \"{user_request}\"")
if mdzen_state.get("model"):
    print(f"  🤖 Model: {mdzen_state['model']}")
else:
    current_model = os.environ.get('MDZEN_CLARIFICATION_MODEL', 'auto-detected')
    print(f"  🤖 Model: {current_model}")
print("=" * 60)
print(f"\n📁 Session: {mdzen_state['session_id']}")
print("\n👉 Run the next cell to get AI clarification questions")

In [ ]:
#@title 🤖 Step 1b: Structure Analysis & Clarification { display-mode: "form" }
#@markdown Analyzes structure and generates clarification questions.

import json
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("user_request"):
    print("❌ Error: Please run Step 1a first")
else:
    user_request = mdzen_state["user_request"]
    session_dir = mdzen_state["session_dir"]
    
    # Reload config to pick up any model changes
    from mdzen import config
    config.settings = config.Settings()
    current_model = config.settings.clarification_model
    
    print("🤖 Starting clarification agent (Streamable HTTP)...")
    print(f"   Model: {current_model}")
    print("-" * 60)
    
    from mdzen.agents.clarification_agent import create_clarification_agent
    from mdzen.tools.mcp_setup import close_toolsets
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    # Use Streamable HTTP transport (more reliable in Colab)
    agent, mcp_tools = create_clarification_agent(transport="http")
    
    session_service = InMemorySessionService()
    runner = Runner(
        app_name="mdzen",
        agent=agent,
        session_service=session_service,
    )
    
    async def run_clarification():
        session = await session_service.create_session(
            app_name="mdzen",
            user_id="colab_user",
            state={"session_dir": session_dir},
        )
        
        message = types.Content(
            role="user",
            parts=[types.Part(text=user_request)],
        )
        
        final_response = None
        async for event in runner.run_async(
            user_id="colab_user",
            session_id=session.id,
            new_message=message,
        ):
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        updated_session = await session_service.get_session(
            app_name="mdzen",
            user_id="colab_user",
            session_id=session.id,
        )
        
        return final_response, updated_session.state
    
    try:
        final_response, session_state = await run_clarification()
        await close_toolsets(mcp_tools)
        
        if session_state.get("simulation_brief"):
            brief = session_state["simulation_brief"]
            if isinstance(brief, str):
                try:
                    brief = json.loads(brief)
                except:
                    pass
            
            if isinstance(brief, dict):
                mdzen_state["simulation_brief"] = brief
                print("\n✅ SimulationBrief Generated!")
                print("-" * 60)
                for key, val in brief.items():
                    if val is not None:
                        print(f"   • {key}: {val}")
                print("\n👉 Proceed to Step 1d to review")
            else:
                mdzen_state["agent_questions"] = brief
                print("\n🤖 Agent needs more information:")
                print("-" * 60)
                print(brief)
                print("\n👉 Answer in Step 1c")
        else:
            if final_response:
                mdzen_state["agent_questions"] = final_response
            print("\n🤖 Agent response:")
            print(final_response or "No response")
            print("\n👉 Answer in Step 1c")
            
    except Exception as e:
        import traceback
        print(f"❌ Error: {e}")
        traceback.print_exc()
        try:
            await close_toolsets(mcp_tools)
        except:
            pass

In [ ]:
#@title 💬 Step 1c: Conversation with Agent { display-mode: "form" }
#@markdown ### Your Response
user_response = "" #@param {type:"string"}
#@markdown Re-run this cell as many times as needed until SimulationBrief is generated.

import json

if 'mdzen_state' not in dir():
    print("❌ Error: Please run Step 1a first")
elif not user_response.strip():
    if mdzen_state.get("simulation_brief") and isinstance(mdzen_state["simulation_brief"], dict):
        print("✅ SimulationBrief already generated! Proceed to Step 1d.")
        brief = mdzen_state["simulation_brief"]
        print(f"   • PDB: {brief.get('pdb_id')} | Chains: {brief.get('select_chains')}")
    elif mdzen_state.get("agent_questions"):
        print("🤖 Agent's questions:")
        print("-" * 50)
        print(mdzen_state["agent_questions"])
        print("-" * 50)
        print("\n👆 Enter your response above and re-run")
    else:
        print("⚠️ Run Step 1b first")
else:
    print(f"💬 Your response: {user_response}")
    print("-" * 50)
    
    # Reload config to pick up any model changes
    from mdzen import config
    config.settings = config.Settings()
    
    from mdzen.agents.clarification_agent import create_clarification_agent
    from mdzen.tools.mcp_setup import close_toolsets
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    session_dir = mdzen_state["session_dir"]
    original_request = mdzen_state.get("user_request", "")
    previous_context = mdzen_state.get("agent_questions", "")
    
    context_message = f"""Original request: {original_request}

Your previous analysis: {previous_context}

User's response: {user_response}

Based on this, either ask follow-up questions OR call generate_simulation_brief with appropriate parameters.
CRITICAL: You must ACTUALLY CALL the tool, not just say you did."""
    
    # Use HTTP transport
    agent, mcp_tools = create_clarification_agent(transport="http")
    session_service = InMemorySessionService()
    runner = Runner(app_name="mdzen", agent=agent, session_service=session_service)
    
    async def run_conversation():
        session = await session_service.create_session(
            app_name="mdzen", user_id="colab_user",
            state={"session_dir": session_dir},
        )
        message = types.Content(role="user", parts=[types.Part(text=context_message)])
        
        print("🔄 Agent thinking...")
        final_response = None
        async for event in runner.run_async(user_id="colab_user", session_id=session.id, new_message=message):
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        updated = await session_service.get_session(app_name="mdzen", user_id="colab_user", session_id=session.id)
        return final_response, updated.state
    
    try:
        final_response, session_state = await run_conversation()
        await close_toolsets(mcp_tools)
        
        if session_state.get("simulation_brief"):
            brief = session_state["simulation_brief"]
            if isinstance(brief, str):
                try: brief = json.loads(brief)
                except: pass
            
            if isinstance(brief, dict):
                mdzen_state["simulation_brief"] = brief
                print("\n" + "=" * 50)
                print("✅ SimulationBrief Generated!")
                print("=" * 50)
                print(f"   • PDB: {brief.get('pdb_id')} | Chains: {brief.get('select_chains')}")
                print(f"   • Temp: {brief.get('temperature')}K | Time: {brief.get('simulation_time_ns')}ns")
                print("\n👉 Proceed to Step 1d")
            else:
                mdzen_state["agent_questions"] = str(brief)
                print("\n🤖 Agent response:")
                print(brief)
        else:
            mdzen_state["agent_questions"] = final_response
            print("\n🤖 Agent response:")
            print(final_response or "No response")
            print("\n👆 Enter response above and re-run")
            
    except Exception as e:
        import traceback
        print(f"❌ Error: {e}")
        traceback.print_exc()

In [ ]:
#@title ✅ Step 1d: Review & Modify SimulationBrief { display-mode: "form" }
#@markdown ### Current SimulationBrief
#@markdown Run this cell to see the current configuration.
#@markdown 
#@markdown ---
#@markdown ### Modifications (optional)
#@markdown Describe any changes you want (leave empty to keep current):
modifications = "" #@param {type:"string"}
#@markdown 
#@markdown **Examples:**
#@markdown - `Change temperature to 310K`
#@markdown - `Use 0.5 ns simulation time`
#@markdown - `Remove pressure (NVT ensemble)`

import json

if 'mdzen_state' not in dir():
    print("❌ Error: Please run Step 1a first")
elif not mdzen_state.get("simulation_brief"):
    print("❌ Error: No SimulationBrief found")
    print("   Please run Step 1b and 1c first to generate the brief.")
else:
    brief = mdzen_state["simulation_brief"]
    
    # Display current brief
    print("📋 Current SimulationBrief:")
    print("=" * 50)
    
    # Group parameters by category
    structure_keys = ['pdb_id', 'fasta_sequence', 'select_chains', 'structure_file']
    ligand_keys = ['ligand_smiles', 'charge_method', 'atom_type']
    solvation_keys = ['water_model', 'box_padding', 'salt_concentration', 'cubic_box', 
                      'cation_type', 'anion_type', 'is_membrane', 'lipids', 'lipid_ratio']
    simulation_keys = ['temperature', 'pressure_bar', 'simulation_time_ns', 'timestep',
                       'minimize_steps', 'nonbonded_cutoff', 'constraints', 'output_frequency_ps']
    forcefield_keys = ['force_field', 'ph', 'cap_termini', 'include_types']
    
    def print_section(title, keys):
        print(f"\n{title}:")
        for key in keys:
            if key in brief and brief[key] is not None:
                val = brief[key]
                if isinstance(val, list):
                    val = ", ".join(str(v) for v in val)
                elif isinstance(val, dict):
                    val = json.dumps(val)
                print(f"  • {key}: {val}")
    
    print_section("📦 Structure", structure_keys)
    print_section("💊 Ligand", ligand_keys)
    print_section("💧 Solvation", solvation_keys)
    print_section("🌡️ Simulation", simulation_keys)
    print_section("⚗️ Force Field", forcefield_keys)
    
    print("\n" + "=" * 50)
    
    # Handle modifications
    if modifications.strip():
        print(f"\n🔄 Applying modifications: {modifications}")
        print("-" * 50)
        
        # Reload config to pick up any model changes
        from mdzen import config
        config.settings = config.Settings()
        
        # Import necessary modules
        from mdzen.agents.clarification_agent import create_clarification_agent
        from mdzen.tools.mcp_setup import close_toolsets
        from google.adk.runners import Runner
        from google.adk.sessions import InMemorySessionService
        from google.genai import types
        
        session_dir = mdzen_state["session_dir"]
        
        # Create agent
        agent, mcp_tools = create_clarification_agent(transport="http")
        session_service = InMemorySessionService()
        runner = Runner(
            app_name="mdzen",
            agent=agent,
            session_service=session_service,
        )
        
        async def apply_modifications():
            session = await session_service.create_session(
                app_name="mdzen",
                user_id="colab_user",
                state={"session_dir": session_dir, "simulation_brief": brief},
            )
            
            # Ask agent to modify the brief
            modify_prompt = f"""The current SimulationBrief is:
{json.dumps(brief, indent=2)}

The user wants to make these modifications:
{modifications}

Please call generate_simulation_brief with the updated parameters.
Keep all other parameters the same unless the user's modification affects them."""
            
            message = types.Content(
                role="user",
                parts=[types.Part(text=modify_prompt)],
            )
            
            final_response = None
            async for event in runner.run_async(
                user_id="colab_user",
                session_id=session.id,
                new_message=message,
            ):
                if event.is_final_response() and event.content:
                    final_response = event.content.parts[0].text if event.content.parts else None
            
            updated_session = await session_service.get_session(
                app_name="mdzen",
                user_id="colab_user",
                session_id=session.id,
            )
            
            return final_response, updated_session.state
        
        try:
            final_response, session_state = await apply_modifications()
            await close_toolsets(mcp_tools)
            
            if session_state.get("simulation_brief"):
                new_brief = session_state["simulation_brief"]
                if isinstance(new_brief, str):
                    try:
                        new_brief = json.loads(new_brief)
                    except:
                        pass
                
                if isinstance(new_brief, dict):
                    mdzen_state["simulation_brief"] = new_brief
                    print("\n✅ SimulationBrief updated!")
                    print("-" * 50)
                    
                    # Show changes
                    for key in new_brief:
                        if key in brief and new_brief[key] != brief[key]:
                            print(f"  ✓ {key}: {brief[key]} → {new_brief[key]}")
                    
                    print("\n👉 Run this cell again to see the full updated brief")
                else:
                    print(f"\n🤖 Agent response: {new_brief[:500] if len(str(new_brief)) > 500 else new_brief}")
            else:
                print(f"\n🤖 Agent response: {final_response[:500] if final_response else 'No response'}")
                
        except Exception as e:
            import traceback
            print(f"\n❌ Error: {e}")
            traceback.print_exc()
            try:
                await close_toolsets(mcp_tools)
            except:
                pass
    else:
        print("\n✅ Ready for Step 2!")
        print("   No modifications requested. Proceed to Step 2 to run the MD workflow.")

---
## Step 2: Run MD Workflow

This will execute all 4 steps automatically:
1. **prepare_complex** - Download structure and prepare proteins/ligands
2. **solvate** - Add water box and ions  
3. **build_topology** - Generate Amber topology files
4. **run_simulation** - Run MD with OpenMM

Click ▶️ to start. Progress will be shown below.

In [ ]:
#@title ⚙️ Step 2: Run Complete Workflow { display-mode: "form" }
#@markdown ### Run Options
run_simulation_step = True #@param {type:"boolean"}
#@markdown > Uncheck to skip the MD simulation (for testing setup only)

import sys
import os
import json
import traceback
import socket
import subprocess
import time
from pathlib import Path

IN_COLAB = 'google.colab' in sys.modules

if 'mdzen_state' not in dir() or not mdzen_state.get("simulation_brief"):
    print("❌ Error: Please run Step 1 first to configure your simulation")
else:
    brief = mdzen_state["simulation_brief"]
    session_dir = Path(mdzen_state["session_dir"])
    
    #==========================================================================
    # Server Health Check and Restart
    #==========================================================================
    # All 6 servers needed by setup agent
    MCP_SERVERS = [
        ("research_server.py", 8001),
        ("structure_server.py", 8002),
        ("genesis_server.py", 8003),
        ("solvation_server.py", 8004),
        ("amber_server.py", 8005),
        ("md_simulation_server.py", 8006),
    ]
    
    def check_port(port, timeout=2):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(timeout)
            result = sock.connect_ex(('localhost', port))
            sock.close()
            return result == 0
        except:
            return False
    
    def restart_server(server, port, python_cmd, server_dir, pythonpath):
        """Start a single MCP server."""
        proc = subprocess.Popen(
            [python_cmd, f"{server_dir}/{server}", "--http", "--port", str(port)],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.PIPE,
            env={**os.environ, "PYTHONPATH": pythonpath},
        )
        return proc
    
    # Check and restart servers
    print("🔍 Checking MCP server health...")
    
    if IN_COLAB:
        PYTHON_CMD = "/usr/local/bin/python"
        SERVER_DIR = "/content/mdzen/servers"
        PYTHONPATH = "/content/mdzen/src"
    else:
        PYTHON_CMD = sys.executable
        SERVER_DIR = "./servers"
        PYTHONPATH = "./src"
    
    servers_restarted = 0
    for server, port in MCP_SERVERS:
        if not check_port(port):
            print(f"   ⚠️ {server} (:{port}) not responding, restarting...")
            restart_server(server, port, PYTHON_CMD, SERVER_DIR, PYTHONPATH)
            servers_restarted += 1
        else:
            print(f"   ✓ {server} (:{port})")
    
    if servers_restarted > 0:
        print(f"   Waiting for {servers_restarted} servers to start...")
        time.sleep(5)
        # Verify all servers are up
        all_up = all(check_port(port) for _, port in MCP_SERVERS)
        if all_up:
            print(f"   ✓ All {len(MCP_SERVERS)} servers ready")
        else:
            print("   ⚠️ Some servers may still be starting, waiting more...")
            time.sleep(5)
    
    #==========================================================================
    # Reload config and run workflow
    #==========================================================================
    from mdzen import config
    config.settings = config.Settings()
    current_model = config.settings.setup_model
    
    print("=" * 60)
    print(f"  🚀 Starting MD Workflow for {brief.get('pdb_id', 'Unknown')}")
    print(f"  🤖 Model: {current_model}")
    print("  📡 Using Streamable HTTP transport")
    print("=" * 60)
    
    from mdzen.agents.setup_agent import create_setup_agent
    from mdzen.tools.mcp_setup import close_toolsets
    
    from google.adk.runners import Runner
    from google.adk.sessions import InMemorySessionService
    from google.genai import types
    
    async def run_setup():
        # Create agent with HTTP transport
        print("\n🔧 Creating setup agent (HTTP)...")
        agent, mcp_tools = create_setup_agent(transport="http")
        
        session_service = InMemorySessionService()
        runner = Runner(
            app_name="mdzen",
            agent=agent,
            session_service=session_service,
        )
        
        initial_state = {
            "session_dir": str(session_dir),
            "simulation_brief": json.dumps(brief) if isinstance(brief, dict) else brief,
            "completed_steps": json.dumps([]),
            "outputs": json.dumps({}),
        }
        
        session = await session_service.create_session(
            app_name="mdzen",
            user_id="colab_user",
            state=initial_state,
        )
        
        steps_to_run = ["prepare_complex", "solvate", "build_topology"]
        if run_simulation_step:
            steps_to_run.append("run_simulation")
        
        request = f"""Execute the MD setup workflow with the following SimulationBrief:

{json.dumps(brief, indent=2)}

Please run these steps in order: {', '.join(steps_to_run)}

Work in the directory: {session_dir}
"""
        
        message = types.Content(
            role="user",
            parts=[types.Part(text=request)],
        )
        
        print("\n🤖 Setup agent is running...")
        print("   (This may take several minutes)")
        print("-" * 60)
        
        final_response = None
        async for event in runner.run_async(
            user_id="colab_user",
            session_id=session.id,
            new_message=message,
        ):
            if event.content and event.content.parts:
                text = event.content.parts[0].text if hasattr(event.content.parts[0], 'text') else None
                if text and not event.is_final_response():
                    if any(kw in text.lower() for kw in ['step', 'complete', 'running', 'preparing', 'building']):
                        print(f"   {text[:200]}...")
                
            if event.is_final_response() and event.content:
                final_response = event.content.parts[0].text if event.content.parts else None
        
        updated_session = await session_service.get_session(
            app_name="mdzen",
            user_id="colab_user",
            session_id=session.id,
        )
        
        return final_response, updated_session.state, mcp_tools
    
    try:
        start_time = time.time()
        final_response, session_state, mcp_tools = await run_setup()
        await close_toolsets(mcp_tools)
        elapsed = time.time() - start_time
        
        outputs = session_state.get("outputs", {})
        if isinstance(outputs, str):
            try: outputs = json.loads(outputs)
            except: outputs = {}
        
        completed = session_state.get("completed_steps", [])
        if isinstance(completed, str):
            try: completed = json.loads(completed)
            except: completed = []
        
        mdzen_state["workflow_outputs"] = outputs
        
        print()
        print("=" * 60)
        print("  🎉 Workflow Complete!")
        print("=" * 60)
        print(f"  ⏱️ Time: {elapsed/60:.1f} min")
        print(f"  ✅ Steps completed: {', '.join(completed) if completed else 'None'}")
        print(f"  📁 Output: {session_dir}")
        
        if outputs:
            print()
            print("  📦 Generated files:")
            for key, path in outputs.items():
                if path:
                    print(f"     • {key}: {Path(path).name if isinstance(path, str) else path}")
        
        print()
        if final_response:
            print("  📝 Agent summary:")
            summary = final_response[:500] + "..." if len(final_response) > 500 else final_response
            for line in summary.split('\n'):
                print(f"     {line}")
        
        print()
        print("  👉 Run the next cell to visualize the trajectory")
        
    except Exception as e:
        print()
        print("=" * 60)
        print(f"  ❌ Error: {e}")
        print("=" * 60)
        print(traceback.format_exc())
        print()
        print("  💡 Tip: If servers crashed, try re-running this cell.")
        print("     The server health check will restart them automatically.")

---
## Step 3: Visualize Results

View the trajectory animation with py3Dmol.

In [ ]:
#@title 🔬 Step 3: Visualize Final Structure { display-mode: "form" }
#@markdown ### Visualization Options
style = "cartoon" #@param ["cartoon", "stick", "sphere", "line"]
show_water = True #@param {type:"boolean"}
#@markdown > Show water molecules (wrapped into periodic box)

import py3Dmol
import tempfile
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("workflow_outputs"):
    print("❌ Error: Please run the workflow first (Step 2)")
elif 'trajectory' not in mdzen_state["workflow_outputs"]:
    print("❌ Error: No trajectory found. Make sure 'Run simulation' was checked in Step 2")
else:
    print("📊 Loading final frame...")
    
    import mdtraj as md
    traj = md.load(
        mdzen_state["workflow_outputs"]['trajectory'], 
        top=mdzen_state["workflow_outputs"]['parm7']
    )
    
    # Get final frame
    final_frame = traj[-1]
    
    # Image molecules (wrap into periodic box)
    final_frame.image_molecules(inplace=True)
    
    print(f"   Trajectory: {traj.n_frames} frames, {traj.time[-1]:.1f} ps total")
    print(f"   Showing: Final frame (t = {final_frame.time[0]:.1f} ps)")
    
    # Select atoms to display
    if show_water:
        # All atoms (protein + water + ions)
        display_frame = final_frame
        atom_info = f"{final_frame.n_atoms} atoms (protein + solvent)"
    else:
        # Protein only
        protein_indices = final_frame.topology.select('protein')
        display_frame = final_frame.atom_slice(protein_indices)
        atom_info = f"{display_frame.n_atoms} protein atoms"
    
    print(f"   Atoms: {atom_info}")
    
    # Save to temporary PDB
    with tempfile.NamedTemporaryFile(suffix='.pdb', delete=False, mode='w') as tmp:
        display_frame.save_pdb(tmp.name, force_overwrite=True)
        tmp_path = tmp.name
    
    # Read PDB content
    with open(tmp_path) as f:
        pdb_content = f.read()
    
    # Create viewer
    view = py3Dmol.view(width=800, height=500)
    view.addModel(pdb_content, 'pdb')
    
    # Apply style based on selection
    if show_water:
        # Protein in cartoon, water as small spheres
        if style == "cartoon":
            view.setStyle({'protein': True}, {'cartoon': {'color': 'spectrum'}})
        elif style == "stick":
            view.setStyle({'protein': True}, {'stick': {}})
        elif style == "sphere":
            view.setStyle({'protein': True}, {'sphere': {'radius': 0.5}})
        else:
            view.setStyle({'protein': True}, {'line': {}})
        
        # Water as small blue spheres
        view.setStyle({'resn': 'WAT'}, {'sphere': {'radius': 0.15, 'color': 'lightblue'}})
        view.setStyle({'resn': 'HOH'}, {'sphere': {'radius': 0.15, 'color': 'lightblue'}})
        
        # Ions as spheres
        view.setStyle({'elem': 'Na'}, {'sphere': {'radius': 0.3, 'color': 'purple'}})
        view.setStyle({'elem': 'Cl'}, {'sphere': {'radius': 0.35, 'color': 'green'}})
    else:
        # Protein only
        if style == "cartoon":
            view.setStyle({'cartoon': {'color': 'spectrum'}})
        elif style == "stick":
            view.setStyle({'stick': {}})
        elif style == "sphere":
            view.setStyle({'sphere': {'radius': 0.5}})
        else:
            view.setStyle({'line': {}})
    
    view.zoomTo()
    
    # Cleanup temp file
    Path(tmp_path).unlink()
    
    print()
    print("✅ Final structure displayed")
    print(f"   Style: {style}" + (" + water" if show_water else ""))
    print()
    print("👉 Run the next cell to download all files")
    
    view.show()


---
## Step 4: Download Results

Download all generated files as a ZIP archive.

In [ ]:
#@title 📥 Step 4: Download Results { display-mode: "form" }

import sys
from pathlib import Path

if 'mdzen_state' not in dir() or not mdzen_state.get("session_dir"):
    print("❌ Error: Please run the workflow first")
else:
    session_dir = Path(mdzen_state["session_dir"])
    
    if not session_dir.exists():
        print("❌ Error: Session directory not found")
    else:
        print("=" * 50)
        print(f"  📂 {session_dir.name}")
        print("=" * 50)
        
        files = sorted(session_dir.rglob('*'))
        total_size = 0
        
        for f in files:
            if f.is_file():
                size = f.stat().st_size
                total_size += size
                size_str = f"{size/1024:.1f} KB" if size > 1024 else f"{size} B"
                print(f"  {f.relative_to(session_dir):<40} {size_str:>10}")
        
        print("=" * 50)
        print(f"  Total: {total_size/1024/1024:.2f} MB")
        print("=" * 50)
        
        if 'google.colab' in sys.modules:
            from google.colab import files
            import shutil
            
            zip_path = f"/content/{session_dir.name}.zip"
            shutil.make_archive(zip_path.replace('.zip', ''), 'zip', session_dir)
            
            print(f"\n⬇️ Downloading {session_dir.name}.zip...")
            files.download(zip_path)
        else:
            print(f"\n📁 Files are in: {session_dir}")